#### Author: Kubam Ivo
#### Date: 7/24/2020


In [2]:
# Import dependencies
import azureml.core
import json

from azureml.core import Workspace
from azureml.core.compute import AksCompute, ComputeTarget
from azureml.core.webservice import AksWebservice

print(azureml.core.VERSION)

1.9.0


In [3]:
# setting up the workspace
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep='\n')

ml_practice
azureml
westeurope
a1839f8e-ad8f-4825-ab97-e9628255f2bb


In [6]:
from azureml.core import Model

model = Model.register(model_path="sklearn_regression_model.pkl", # This points to a local file.
                       model_name="sklearn_regression_model.pkl", # This is the name the model is registered as.
                       tags={'area': "diabetes", 'type': "regression"},
                       description="Ridge regression model to predict diabetes",
                       workspace=ws)

print(model.name, model.description, model.version)

[Model(workspace=Workspace.create(name='ml_practice', subscription_id='a1839f8e-ad8f-4825-ab97-e9628255f2bb', resource_group='azureml'), name=my_sklearn-model, id=my_sklearn-model:3, version=3, tags={'area': 'diabestes', 'type': 'regression'}, properties={})]

In [ ]:
# The score.py is updated with the print statement

In [7]:
# create myenv.yml file
from azureml.core.conda_dependencies import CondaDependencies
myenv = CondaDependencies.create(conda_packages=['numpy', 'scikit-learn'],
                                pip_packages=['azureml-defaults'])

with open('myenv.yml', 'w') as f:
    f.write(myenv.serialize_to_string())

In [8]:
# create an inference configuration
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig

myenv = Environment.from_conda_specification(name='myenv', file_path="myenv.yml")
inference_config = InferenceConfig(entry_script='score.py', environment=myenv)

In [9]:
#Deploy to ACI
from azureml.core.webservice import AciWebservice

aci_deployment_config = AciWebservice.deploy_configuration(cpu_cores=1,
                                                            memory_gb=1,
                                                            tags={'area': 'diabetes', 'type':'regression'},
                                                            description = 'Predict diabetes using regression model',
                                                            enable_app_insights=True)

In [10]:

aci_service_name = "aci-service-appinsights"

aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aci_deployment_config, overwrite=True)
aci_service.wait_for_deployment(show_output=True)

print(aci_service.state)

NameError: name 'model' is not defined